In [1]:
import datetime
import pandas as pd
import glob
import numpy as np
from sqlalchemy import create_engine
from helper_functions import save_data, load_data

## Reading the stations

First load the file `stations.csv` to see what is inside. Therefore i loaded a stations file of the last month. Here a short description of the information inside:

`uuid,name,brand,street,house_number,post_code,city,latitude,longitude, first_active, openingtimes_json`



In [2]:
# change the directory as necessary
stations_pd = pd.read_csv('e:/Working/sprit/stations.csv')

In [3]:
stations_pd.head()

,uuid,name,brand,street,house_number,post_code,city,latitude,longitude,first_active,openingtimes_json
0,ad812258-94e7-473d-aa80-d392f7532218,bft Bonn-Bad Godesberg,bft,Godesberger Allee,55,53175,Bonn,50.695100,7.142760,1970-01-01 01:00:00+01,"{""openingTimes"":[{""applicable_days"":32,""period..."
1,44e2bdb7-13e3-4156-8576-8326cdd20459,bft Tankstelle,NaN,Schellengasse,53,36304,Alsfeld,50.752010,9.279039,1970-01-01 01:00:00+01,"{""openingTimes"":[{""applicable_days"":63,""period..."
2,0e18d0d3-ed38-4e7f-a18e-507a78ad901d,OIL! Tankstelle München,OIL!,Eversbuschstraße 33,NaN,80999,München,48.180700,11.460900,1970-01-01 01:00:00+01,"{""openingTimes"":[{""applicable_days"":63,""period..."
3,db0655ed-4b94-4c12-a75d-26b2d6a38759,AVIA Albstadt-Ebingen,AVIA,Lautlinger Str.,3,72458,Albstadt,48.212800,9.012450,2014-03-18 16:45:31+01,"{""openingTimes"":[{""applicable_days"":63,""period..."
4,6a625a13-310c-43fb-8316-4e8027a54510,Aral Tankstelle,ARAL,Leipziger Straße,208,08058,Zwickau,50.742687,12.488169,2014-03-18 16:45:31+01,{}


In [4]:
stations_pd.shape

(15859, 11)

In [5]:
stations_pd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15859 entries, 0 to 15858
Data columns (total 11 columns):
uuid                 15859 non-null object
name                 15859 non-null object
brand                15243 non-null object
street               15856 non-null object
house_number         12014 non-null object
post_code            15856 non-null object
city                 15855 non-null object
latitude             15859 non-null float64
longitude            15859 non-null float64
first_active         15859 non-null object
openingtimes_json    15859 non-null object
dtypes: float64(2), object(9)
memory usage: 1.3+ MB


In [6]:
# want to see all stations around zip-code 40xxx
short_pd = stations_pd.dropna(subset=['post_code'])
short_pd = short_pd[short_pd['post_code'].str.match(pat = '40\d{3}')]

## Reading Prices

The prices are organized in folders for every year and month and is really big data (around 5 GB per year). Therefore i've build a function to load the Data per year and extract only a few data for stations in short_pd. The pricesxxx.csv are in the following format:

`date,station_uuid,diesel,e5,e10,dieselchange,e5change,e10change`

In [7]:
# let's read one file to see special cases or wrong data
prices_tmp = pd.read_csv('e:Working/sprit/2015/01/2015-01-01-prices.csv', usecols=['date', 'station_uuid', 'diesel', 'e5', 'e10'])

In [8]:
prices_tmp.head()

,date,station_uuid,diesel,e5,e10
0,2015-01-01 00:02:01+01,00060151-0001-4444-8888-acdc00000001,1.189,1.329,1.289
1,2015-01-01 00:02:01+01,005056ba-7cb6-1ed2-bceb-6e6ee17d4d20,1.179,1.319,1.279
2,2015-01-01 00:02:01+01,139897a8-a161-4f4c-b550-bca985affa38,1.159,1.299,1.259
3,2015-01-01 00:02:01+01,18aef2d2-ed53-44d2-9f7f-512f994593c2,1.219,1.359,1.319
4,2015-01-01 00:02:01+01,3e7c0f12-9665-40e2-7c70-d0dfb19314cd,1.229,1.369,1.329


In [9]:
prices_tmp.describe()

,diesel,e5,e10
count,53868.000000,53868.000000,53868.000000
mean,1.181945,1.292544,1.256904
std,0.051318,0.221374,0.215966
min,-0.001000,-0.001000,-0.001000
25%,1.149000,1.289000,1.259000
50%,1.179000,1.329000,1.289000
75%,1.219000,1.359000,1.329000
max,2.015000,2.015000,2.015000


In [10]:
prices_tmp[~(prices_tmp.diesel < 0.5)].describe()

,diesel,e5,e10
count,53853.000000,53853.000000,53853.000000
mean,1.182274,1.292632,1.256990
std,0.047381,0.221122,0.215722
min,1.009000,-0.001000,-0.001000
25%,1.149000,1.289000,1.259000
50%,1.179000,1.329000,1.289000
75%,1.219000,1.359000,1.329000
max,2.015000,2.015000,2.015000


Min Price of -0.001 is not a real value, so i will replace it with np.nan. Furthermor 2.015 € for all types are max? Let's have a look at this also...

In [11]:
prices_tmp[prices_tmp.diesel > 1.70].head()

,date,station_uuid,diesel,e5,e10
24,2015-01-01 00:06:01+01,42d6d4cc-6909-45fc-f60f-a1abd9ea1f0e,2.015,2.015,2.015


In [12]:
prices_tmp[prices_tmp['station_uuid'] == '42d6d4cc-6909-45fc-f60f-a1abd9ea1f0e']

,date,station_uuid,diesel,e5,e10
24,2015-01-01 00:06:01+01,42d6d4cc-6909-45fc-f60f-a1abd9ea1f0e,2.015,2.015,2.015


It seems like this station has made an joke for new year ;-) This dataset will be removed later, after all prices are read. 

In [13]:
# let's read another file to see special cases or wrong data
prices_tmp = pd.read_csv('e:Working/sprit/2016/01/2016-01-01-prices.csv', usecols=['date', 'station_uuid', 'diesel', 'e5', 'e10'])

In [14]:
prices_tmp.describe()

,diesel,e5,e10
count,85708.000000,85708.000000,85708.000000
mean,1.034316,1.256426,1.236780
std,0.053128,0.209600,0.207974
min,-0.001000,-0.001000,-0.001000
25%,0.999000,1.249000,1.229000
50%,1.029000,1.279000,1.259000
75%,1.059000,1.309000,1.289000
max,2.028000,1.619000,2.000000


In [15]:
prices_tmp[(prices_tmp.diesel <= 1.70) & (prices_tmp.diesel > 0.5)].describe()

,diesel,e5,e10
count,85691.000000,85691.000000,85691.000000
mean,1.034498,1.256527,1.236836
std,0.051106,0.209312,0.207822
min,0.908000,-0.001000,-0.001000
25%,0.999000,1.249000,1.229000
50%,1.029000,1.279000,1.259000
75%,1.059000,1.309000,1.289000
max,1.269000,1.619000,2.000000


### Reading and cleaning data

In the above examples i've found some wrong data in this two files. After reading the whole dataset i'll have a look on the statistics to see what can be dropped.

In [16]:

def read_data_year(year, filepath):
    '''
    read all relevant data for year x
    Input: year, filepath with 
    Output: dataframe with all prices for region 40x and year
    '''
    print('Read year {}'.format(year))
    
    files = glob.glob('{}{}/*/*'.format(filepath, year))
    prices = [pd.read_csv(file, usecols=['date', 'station_uuid', 'diesel', 'e5', 'e10']) for file in files]
    prices_pd = pd.concat(prices, sort=False)
    data_pd = prices_pd[prices_pd['station_uuid'].isin(short_pd['uuid'])][['date', 'station_uuid', 'diesel', 'e5', 'e10']]
    return data_pd

def read_data(filepath='e:/Working/sprit/'):
    '''
    read all data and make some conversions
    Input: filepath to input files, default is given
    Output: dataframe with datetimeindex and station_uuid as category
    '''
    prices = read_data_year('2015', filepath)
    prices = prices.append(read_data_year('2016', filepath))
    prices = prices.append(read_data_year('2017', filepath))
    prices = prices.append(read_data_year('2018', filepath))
    prices = prices.append(read_data_year('2019', filepath))

    # replace wrong data (<0.5 here for small) with nan 
    prices['diesel'] = prices['diesel'].apply(lambda x: np.nan if x < 0.5 else x)
    prices['e5'] = prices['e5'].apply(lambda x: np.nan if x < 0.5 else x)
    prices['e10'] = prices['e10'].apply(lambda x: np.nan if x < 0.5 else x)
    prices.station_uuid = prices.station_uuid.astype('category')
    prices.date = pd.to_datetime(prices.date, utc=True)
    return prices

In [17]:
# Reading of all relevant data
prices_pd = read_data()

Read year 2015
Read year 2016
Read year 2017
Read year 2018
Read year 2019


In [18]:
prices_pd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2677793 entries, 74 to 248042
Data columns (total 5 columns):
date            datetime64[ns, UTC]
station_uuid    category
diesel          float64
e5              float64
e10             float64
dtypes: category(1), datetime64[ns, UTC](1), float64(3)
memory usage: 107.3 MB


In [19]:
prices_pd.describe()

,diesel,e5,e10
count,2.676682e+06,2.674904e+06,2.654523e+06
mean,1.193693e+00,1.387779e+00,1.363561e+00
std,9.971780e-02,8.483290e-02,8.389918e-02
min,8.790000e-01,9.090000e-01,1.029000e+00
25%,1.129000e+00,1.329000e+00,1.309000e+00
50%,1.199000e+00,1.379000e+00,1.359000e+00
75%,1.259000e+00,1.449000e+00,1.429000e+00
max,8.888000e+00,2.918000e+00,2.878000e+00


The max values of the fuel types differs too much from the 75% percentile, the values are probable not useful. Let's see which values we can set to NaN. I'll found from looking into the data following values useful.

In [20]:
prices_pd[prices_pd.diesel > 1.7]

,date,station_uuid,diesel,e5,e10
36871,2015-04-26 12:46:01+00:00,005056ba-7cb6-1ed2-bceb-7e82e4910d2a,2.458,2.918,2.878
30833,2015-07-16 08:06:01+00:00,ee34d5a4-be1a-2a3a-e040-0b0a3dfe5d3f,8.888,1.469,1.449
33012,2015-07-16 08:26:01+00:00,ee34d5a4-be1a-2a3a-e040-0b0a3dfe5d3f,8.888,1.429,1.409
55772,2015-07-16 10:42:01+00:00,ee34d5a4-be1a-2a3a-e040-0b0a3dfe5d3f,8.888,1.459,1.439
8513,2016-02-08 05:34:01+00:00,67765d63-fec8-4545-9c09-940ea88a8566,1.979,1.239,1.219
116369,2019-03-25 13:41:06+00:00,b253485f-535c-4554-ba4e-9c2c7093b8cc,1.999,1.339,1.319
181207,2019-04-13 15:51:06+00:00,b253485f-535c-4554-ba4e-9c2c7093b8cc,1.999,1.399,1.379
46421,2019-08-02 06:53:05+00:00,09ee977e-c04e-4db1-98bf-b507b123423c,1.999,1.399,1.379
51459,2019-11-05 08:05:04+00:00,b253485f-535c-4554-ba4e-9c2c7093b8cc,1.779,1.339,1.319


In [21]:
prices_pd[prices_pd.e10 > 1.75]

,date,station_uuid,diesel,e5,e10
36871,2015-04-26 12:46:01+00:00,005056ba-7cb6-1ed2-bceb-7e82e4910d2a,2.458,2.918,2.878


In [22]:
prices_pd[prices_pd.e5 > 1.82]

,date,station_uuid,diesel,e5,e10
36871,2015-04-26 12:46:01+00:00,005056ba-7cb6-1ed2-bceb-7e82e4910d2a,2.458,2.918,2.878
43036,2016-03-24 09:47:02+00:00,09ee977e-c04e-4db1-98bf-b507b123423c,0.969,1.889,1.169


In [23]:
prices_pd['diesel'] = prices_pd['diesel'].apply(lambda x: np.nan if x > 1.7 else x)
prices_pd['e10'] = prices_pd['e10'].apply(lambda x: np.nan if x > 1.75 else x)
prices_pd['e5'] = prices_pd['e5'].apply(lambda x: np.nan if x > 1.82 else x)

In [24]:
prices_pd.isna().sum()

date                0
station_uuid        0
diesel           1120
e5               2891
e10             23271
dtype: int64

In [25]:
#drop all rows where all fuel values are NaN
prices_pd.dropna(axis=0, how='all', subset=['diesel', 'e5', 'e10'], inplace=True)

In [26]:
prices_pd.isna().sum()

date                0
station_uuid        0
diesel            948
e5               2719
e10             23099
dtype: int64

In [27]:
prices_pd.describe()

,diesel,e5,e10
count,2.676673e+06,2.674902e+06,2.654522e+06
mean,1.193682e+00,1.387779e+00,1.363561e+00
std,9.937624e-02,8.482722e-02,8.389405e-02
min,8.790000e-01,9.090000e-01,1.029000e+00
25%,1.129000e+00,1.329000e+00,1.309000e+00
50%,1.199000e+00,1.379000e+00,1.359000e+00
75%,1.259000e+00,1.449000e+00,1.429000e+00
max,1.639000e+00,1.819000e+00,1.719000e+00


Now the data looks homogenous and we can save it to the sqllite database.

### Run the next cell only to reduce data set

In [28]:
# for reducing the data to upload it in github, i'll choose only 8 stations

uuids = ['005056ba-7cb6-1ed2-bceb-82ea369c0d2d', '79fb1f24-bebb-489e-841f-728f9053b555', '51d4b59c-a095-1aa0-e100-80009459e03a', '005056ba-7cb6-1ed2-bceb-a46e32000d3e', 
         'e43c09b0-5baa-4738-962a-c94388e93c30', '82119d5d-775f-42af-ac56-000d7c91413f', 'e7807347-796f-4aac-997d-07d0c988e109', '5bf85d09-ea6b-4146-b23f-4b902e2e1554']

prices_pd = prices_pd[prices_pd['station_uuid'].isin(uuids)][['date', 'station_uuid', 'diesel', 'e5', 'e10']]


### Save the data to sqllite file, it will overwrite the tables

In [29]:
# run this cell only if you read new csv files, it will overwrite the sql file 

save_data(prices_pd, 'Prices', './Data/prices_40.sql')
save_data(stations_pd, 'Stations', './Data/prices_40.sql')